In [19]:
from sys import path
path.append("../")

In [20]:
import jax
import jax.numpy as jnp
from flax.struct import dataclass 

from abc import abstractmethod

In [21]:
from fg import Gaussian

# Inter Robot

In [22]:
x = jnp.array([
    [1,1],
    [2,2],
    [3,3],
    [4,4]
])
states = jnp.stack((x, x + 0.1, x + 0.3))
vels = jnp.zeros_like(states)
states = jnp.concat((states, vels), axis=-1)
states, states.shape

(Array([[[1. , 1. , 0. , 0. ],
         [2. , 2. , 0. , 0. ],
         [3. , 3. , 0. , 0. ],
         [4. , 4. , 0. , 0. ]],
 
        [[1.1, 1.1, 0. , 0. ],
         [2.1, 2.1, 0. , 0. ],
         [3.1, 3.1, 0. , 0. ],
         [4.1, 4.1, 0. , 0. ]],
 
        [[1.3, 1.3, 0. , 0. ],
         [2.3, 2.3, 0. , 0. ],
         [3.3, 3.3, 0. , 0. ],
         [4.3, 4.3, 0. , 0. ]]], dtype=float32, weak_type=True),
 (3, 4, 4))

In [23]:
def find_closest_robot(states: jnp.ndarray):
    def find_closest_robot_across_horizon(robot, other_robots):
        closest_index = jnp.argmin(jnp.linalg.norm(robot[0:2] - other_robots[:,0:2], axis=1))
        return closest_index

    def find_batched_closest_robot(batch_states, i):
        modified_states = batch_states.at[i].set(jnp.inf)
        return jax.vmap(find_closest_robot_across_horizon, in_axes=(0, 1))(batch_states[i], modified_states)
    return jax.vmap(find_batched_closest_robot, in_axes=(None, 0))(states, jnp.arange(states.shape[0]))
closest_robots = find_closest_robot(states) # ideally, should be # (N - 1, 4, 2)
closest_robots

Array([[1, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1]], dtype=int32)

In [38]:
N_STATES = 4
POSE_NOISE = 1e-15
DYNAMICS_NOISE = 0.005
OBSTACLE_NOISE = 0.005
INTER_ROBOT_NOISE = 0.005

class Factor:
    def __init__(
        self, state: jnp.array, state_precision: jnp.ndarray, dims: jnp.array, linear: bool = True
    ) -> None:
        self._state = state
        self._state_precision = state_precision
        self._linear = linear
        self._dims = dims

    def calculate_likelihood(self) -> Gaussian:
        return Gaussian(
            self._calc_info(self._state, self._state_precision),
            self._calc_precision(self._state, self._state_precision),
            self._dims
        )

    @abstractmethod
    def _calc_measurement(self, state: jnp.ndarray) -> jnp.ndarray:
        pass

    def _calc_info(self, state: jnp.ndarray, precision: jnp.ndarray) -> jnp.ndarray:
        X = state
        if self._linear:
            eta = precision @ (jnp.zeros(N_STATES) - self._calc_measurement(state))
        else:
            J = jax.jacfwd(self._calc_measurement)(state)
            eta = (J.T @ precision) @ (
                (J @ X.reshape((-1,1))) + 0 - self._calc_measurement(state).reshape((-1,1))
            )
        return eta.squeeze()

    def _calc_precision(self, state: jnp.ndarray, precision: jnp.ndarray) -> jnp.ndarray:
        if self._linear:
            return precision
        else:
            J = jax.jacfwd(self._calc_measurement)(state)
            return J.T @ precision @ J

class InterRobotFactor:
    def __init__(
        self,
        state: jnp.ndarray,
        critical_distance: float,
        t: jnp.ndarray, #ndarray just to hold time,
        dims: jnp.ndarray,
    ) -> None:
        self._crit_distance = critical_distance
        self._z_precision = 100

        self._dist = self._calc_dist(state[0:4], state[4:])
        dx, dy = (state[0] - state[4])/self._dist, (state[1] - state[5])/self._dist
        self._J = jnp.array([[-dx/self._crit_distance, -dy/self._crit_distance, 0, 0,
                              dx/self._crit_distance, dy/self._crit_distance, 0, 0]])

        self._state = state
        # self._precision = jnp.pow(t * INTER_ROBOT_NOISE, -2) * jnp.eye(N_STATES)
        self._state_precision = self._z_precision * jnp.eye(1)
        self._dims = dims
    
    def calculate_likelihood(self) -> Gaussian:
        return Gaussian(
            self._calc_info(self._state, self._state_precision),
            self._calc_precision(self._state, self._state_precision),
            self._dims
        )
 
    def _calc_dist(self, state: jnp.array, other_state: jnp.array):
        return jnp.linalg.norm(state[0:2] - other_state[0:2]) 
    
    def _calc_info(self, state: jnp.ndarray, state_precision: jnp.ndarray) -> jnp.ndarray:
        def safe_fn():
            return (state_precision @ state[jnp.newaxis,:]).squeeze()
        def unsafe_fn():
            return (self._J.T @ state_precision @ (self._J @ state[:,jnp.newaxis] - self._calc_measurement(state))).squeeze()
        info = jax.lax.select(self._dist >= self._crit_distance, safe_fn(), unsafe_fn())
        return info
        
    def _calc_precision(self, state: jnp.ndarray, state_precision: jnp.ndarray) -> jnp.ndarray:
        unsafe_precision = self._J.T @ state_precision @ self._J 
        unsafe_precision = unsafe_precision.at[2:4,2:4].set(jnp.eye(2)).at[6:,6:].set(jnp.eye(2))

        # Update A
        unsafe_precision = unsafe_precision.at[:2,:2].set(unsafe_precision[0, 0])

        # Update B
        unsafe_precision = unsafe_precision.at[4:6,4:6].set(unsafe_precision[4,4])

        # Update C
        unsafe_precision = unsafe_precision.at[0:2, 4:6].set(unsafe_precision[0, 4])

        # Update D
        unsafe_precision = unsafe_precision.at[4:6, 0:2].set(unsafe_precision[4, 0])

        precision = jax.lax.select(self._dist >= self._crit_distance, jnp.eye(8), unsafe_precision)
        return precision @ precision.T

    def _calc_measurement(self, state: jnp.ndarray):
        current_state = state[0:4]
        other_state = state[4:]
        dist = self._calc_dist(current_state, other_state)
        measurement = jax.lax.select(
            dist < self._crit_distance, 1.0 - dist / self._crit_distance, 0.
        )
        return measurement

In [39]:
ir_factor =  InterRobotFactor(jnp.array([-5.0, 0., 0.5, 0.0, 5.0, 0.0, -0.5, 0.0]), 1.5, jnp.array([1.]), jnp.array([1., 1., 1., 1., 100., 100., 100., 100.]))
g = ir_factor.calculate_likelihood()
g

Gaussian(info=Array([-500.,    0.,   50.,    0.,  500.,    0.,  -50.,    0.], dtype=float32), precision=Array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32), dims=Array([  1.,   1.,   1.,   1., 100., 100., 100., 100.], dtype=float32))

In [32]:
g = g.marginalize(jnp.array([1., 1., 1., 1.]))
g

Gaussian(info=Array([500.,   0., -50.,   0.], dtype=float32), precision=Array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32), dims=Array([100., 100., 100., 100.], dtype=float32))

In [33]:
jnp.linalg.inv(g.precision)

Array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)

In [34]:
ir_factor =  InterRobotFactor(jnp.array([-0.1, 0., 0.1, 0.0, 0.1, 0.0, -0.1, 0.0]), 1.5, jnp.array([1.]), jnp.array([1., 1., 1., 1., 100., 100., 100., 100.]))
g = ir_factor.calculate_likelihood()
g

Gaussian(info=Array([-66.66667,   0.     ,  -0.     ,  -0.     ,  66.66667,  -0.     ,
        -0.     ,  -0.     ], dtype=float32), precision=Array([[ 3.9506184e+03,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        -3.9506184e+03,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  1.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-3.9506184e+03,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         3.9506184e+03,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0

In [35]:
g = g.marginalize(jnp.array([1., 1., 1., 1.]))
g

Gaussian(info=Array([0., 0., 0., 0.], dtype=float32), precision=Array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32), dims=Array([100., 100., 100., 100.], dtype=float32))

In [37]:
jnp.linalg.inv(g.precision)

Array([[nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan]], dtype=float32)

# Obstacle

In [12]:
# obstacle = jnp.array([
#     [4.2, 4.2],
#     [1.0, 1.0]
# ])

# def find_closest_obstacle(states, obstacles):
#     def find_closest_obstacle_for_pt(state_t, obstacles):
#         closest_obstacle_idx = jnp.argmin(jnp.linalg.norm(state_t - obstacles, axis=1))
#         return obstacles[closest_obstacle_idx]
#     def batch_find_closest_obstacle(agent_states, obstacles):
#         return jax.vmap(find_closest_obstacle_for_pt, in_axes=(0, None))(agent_states, obstacles)
#     return jax.vmap(batch_find_closest_obstacle, in_axes=(0, None))(states, obstacles)
# closest_obstacle = find_closest_obstacle(states, obstacle)
# closest_obstacle, closest_obstacle.shape

# Random

In [13]:
x = jnp.array([[1,2,3,4,5,6], [7,8,9,10,11,12]])
y = x * 2
x + y

Array([[ 3,  6,  9, 12, 15, 18],
       [21, 24, 27, 30, 33, 36]], dtype=int32)

In [14]:
x = jnp.arange(32).reshape((8,4))
x[1:-1:2] + x[2:-1:2]

Array([[12, 14, 16, 18],
       [28, 30, 32, 34],
       [44, 46, 48, 50]], dtype=int32)

In [15]:
x[1:-1:2], x[2:-1:2]

(Array([[ 4,  5,  6,  7],
        [12, 13, 14, 15],
        [20, 21, 22, 23]], dtype=int32),
 Array([[ 8,  9, 10, 11],
        [16, 17, 18, 19],
        [24, 25, 26, 27]], dtype=int32))

In [18]:
x = jnp.eye(8)
A = x[0:2,0:2]
B = x[0:2,4:6]
C = x[4:6,0:2]
D = x[4:6,4:6]
x = x.at[0:2,0:2].set(jnp.where(A == 0, 1.0, A))
x = x.at[0:2,4:6].set(jnp.where(B == 0, 1.0, B))
x = x.at[4:6,0:2].set(jnp.where(C == 0, 1.0, C))
x = x.at[4:6,4:6].set(jnp.where(D == 0, 1.0, D))
x, jnp.linalg.inv(x)

(Array([[1., 1., 0., 0., 1., 1., 0., 0.],
        [1., 1., 0., 0., 1., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 1., 1., 0., 0.],
        [1., 1., 0., 0., 1., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32),
 Array([[nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]], dtype=float32))